<a href="https://colab.research.google.com/github/AnshulH/NLP-DL-Group2/blob/main/NLP_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -r /content/NLP-DL-Group2/
!cd /content && git clone git@github.com:AnshulH/NLP-DL-Group2.git

rm: cannot remove '/content/NLP-DL-Group2/': No such file or directory
Cloning into 'NLP-DL-Group2'...
Host key verification failed.
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


In [ ]:
!python3 /content/NLP-DL-Group2/hw#1/starter.py

vocab: 44223 train: 3366260
vocab: 44223 test: 3366260
Namespace(batch_size=20, clip=2.0, d_hidden=100, d_model=100, dropout=0.35, epochs=20, loadname=None, lr=0.0001, model='LSTM', n_layers=2, printevery=5000, savename='lstm', seq_len=30, testname='/content/NLP-DL-Group2/hw#1/mix.train.txt', trainname='/content/NLP-DL-Group2/hw#1/mix.train.txt', validname='/content/NLP-DL-Group2/hw#1/mix.train.txt', vocab_size=44223, window=3)
